In [51]:
from linearmodels import PanelOLS
from statsmodels.iolib.summary2 import summary_col
import pandas as pd
import numpy as np

In [45]:
df = pd.read_csv('processed.csv')
df['const'] = 1
print(len(np.unique(df['MSA'])))
print(len(np.unique(df['Year'])))

105
19


In [46]:
df = df.set_index(['MSA', 'Year'])
df

Sex Sex Code   Race Race Code  Year Code  \
MSA            Year                                               
Akron, OH      1999.0    0        F  White    2106-3     1999.0   
               2000.0    0        F  White    2106-3     2000.0   
               2001.0    0        F  White    2106-3     2001.0   
               2002.0    0        F  White    2106-3     2002.0   
               2003.0    0        F  White    2106-3     2003.0   
...                    ...      ...    ...       ...        ...   
Youngstown, OH 2013.0    1        M  White    2106-3     2013.0   
               2014.0    1        M  White    2106-3     2014.0   
               2015.0    1        M  White    2106-3     2015.0   
               2016.0    1        M  White    2106-3     2016.0   
               2017.0    1        M  White    2106-3     2017.0   

                                 Leading Cancer Sites  \
MSA            Year                                     
Akron, OH      1999.0  Brain and Other Nervous System   
               2000.0  Brain and Other Nervous System   
               2001.0  Brain and Other Nervous System   
               2002.0  Brain and Other Nervous System   
               2003.0  Brain and Other Nervous System   
...                                               ...   
Youngstown, OH 2013.0  Brain and Other Nervous System   
               2014.0  Brain and Other Nervous System   
               2015.0  Brain and Other Nervous System   
               2016.0  Brain and Other Nervous System   
               2017.0  Brain and Other Nervous System   

                      Leading Cancer Sites Code  Count  Population  \
MSA            Year                                                  
Akron, OH      1999.0               31010-31040   17.0      310511   
               2000.0               31010-31040   26.0      310857   
               2001.0               31010-31040   17.0      310845   
               2002.0               31010-31040   24.0      310777   
               2003.0               31010-31040   22.0      310310   
...                                         ...    ...         ...   
Youngstown, OH 2013.0               31010-31040   26.0      236173   
               2014.0               31010-31040   23.0      235027   
               2015.0               31010-31040   31.0      233088   
               2016.0               31010-31040   26.0      231337   
               2017.0               31010-31040   20.0      230116   

                       Crude Rate   latitude  longitude   min_dist  \
MSA            Year                                                  
Akron, OH      1999.0         5.5  41.083064 -81.518485  53.201224   
               2000.0         8.4  41.083064 -81.518485  53.201224   
               2001.0         5.5  41.083064 -81.518485  53.201224   
               2002.0         7.7  41.083064 -81.518485  53.201224   
               2003.0         7.1  41.083064 -81.518485  53.201224   
...                           ...        ...        ...        ...   
Youngstown, OH 2013.0        11.0  41.103579 -80.652016  35.152136   
               2014.0         9.8  41.103579 -80.652016  35.152136   
               2015.0        13.3  41.103579 -80.652016  35.152136   
               2016.0        11.2  41.103579 -80.652016  35.152136   
               2017.0         8.7  41.103579 -80.652016  35.152136   

                       min_dist_squared  const  
MSA            Year                             
Akron, OH      1999.0       2830.370286    1.0  
               2000.0       2830.370286    1.0  
               2001.0       2830.370286    1.0  
               2002.0       2830.370286    1.0  
               2003.0       2830.370286    1.0  
...                                 ...    ...  
Youngstown, OH 2013.0       1235.672662    1.0  
               2014.0       1235.672662    1.0  
               2015.0       1235.672662    1.0  
               2016.0       1235.672662    1.0  

In [72]:
def pprint(results):
    for result in results:
        print(round(result.params.values[0], 4), round(result.std_errors.values[0], 4))

In [76]:
mod = PanelOLS(df['Crude Rate'], df[['min_dist']], entity_effects=True, time_effects=False)
entity = mod.fit(cov_type='robust')
mod = PanelOLS(df['Crude Rate'], df[['min_dist']], entity_effects=False, time_effects=True)
time = mod.fit(cov_type='robust')
mod = PanelOLS(df['Crude Rate'], df[['min_dist']], entity_effects=True, time_effects=True)
both = mod.fit(cov_type='robust')
pprint([entity, time, both])

-0.0023 310741275662.3998
-0.0021 0.0003
-0.0022 311134029423.8452
